In [1]:
data_path = "src/resources/data/movie.csv"

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("customer_look_alike_modelling").getOrCreate()
df = spark.read.option("header", "true").csv(data_path)
df.limit(5).show()

24/08/26 11:59:49 WARN Utils: Your hostname, dvadmin-ThinkPad-E480 resolves to a loopback address: 127.0.1.1; using 192.168.0.15 instead (on interface wlp5s0)
24/08/26 11:59:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/26 12:00:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/26 12:00:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/08/26 12:00:10 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


+---+-------------+-----------------+-------------+--------------------------+----------------+------+
|Age|FrequentFlyer|AnnualIncomeClass|ServicesOpted|AccountSyncedToSocialMedia|BookedHotelOrNot|Target|
+---+-------------+-----------------+-------------+--------------------------+----------------+------+
| 34|           No|    Middle Income|            6|                        No|             Yes|     0|
| 34|          Yes|       Low Income|            5|                       Yes|              No|     1|
| 37|           No|    Middle Income|            3|                       Yes|              No|     0|
| 30|           No|    Middle Income|            2|                        No|              No|     0|
| 30|           No|       Low Income|            1|                        No|              No|     0|
+---+-------------+-----------------+-------------+--------------------------+----------------+------+



In [3]:
from pyspark.sql import functions as F
df = df.withColumnRenamed("BookedHotelOrNot", "BookedFoodOrNot")
df = df.withColumnRenamed("FrequentFlyer", "FrequentWatcher")
df = df.withColumn("Id", F.monotonically_increasing_id())
df = df.withColumn("temp", F.col("ServicesOpted") + 2)
df = df.drop("ServicesOpted").withColumn("ServicesOpted", F.col("temp")).drop("temp")
df.limit(20).show()

+---+---------------+-----------------+--------------------------+---------------+------+---+-------------+
|Age|FrequentWatcher|AnnualIncomeClass|AccountSyncedToSocialMedia|BookedFoodOrNot|Target| Id|ServicesOpted|
+---+---------------+-----------------+--------------------------+---------------+------+---+-------------+
| 34|             No|    Middle Income|                        No|            Yes|     0|  0|          8.0|
| 34|            Yes|       Low Income|                       Yes|             No|     1|  1|          7.0|
| 37|             No|    Middle Income|                       Yes|             No|     0|  2|          5.0|
| 30|             No|    Middle Income|                        No|             No|     0|  3|          4.0|
| 30|             No|       Low Income|                        No|             No|     0|  4|          3.0|
| 27|            Yes|      High Income|                        No|            Yes|     1|  5|          3.0|
| 34|             No|    Mid

In [4]:
df = df.withColumn("temp", F.col("ServicesOpted").cast("int"))
df = df.drop("ServicesOpted").withColumn("ServicesOpted", F.col("temp")).drop("temp")
df.show()

+---+---------------+-----------------+--------------------------+---------------+------+---+-------------+
|Age|FrequentWatcher|AnnualIncomeClass|AccountSyncedToSocialMedia|BookedFoodOrNot|Target| Id|ServicesOpted|
+---+---------------+-----------------+--------------------------+---------------+------+---+-------------+
| 34|             No|    Middle Income|                        No|            Yes|     0|  0|            8|
| 34|            Yes|       Low Income|                       Yes|             No|     1|  1|            7|
| 37|             No|    Middle Income|                       Yes|             No|     0|  2|            5|
| 30|             No|    Middle Income|                        No|             No|     0|  3|            4|
| 30|             No|       Low Income|                        No|             No|     0|  4|            3|
| 27|            Yes|      High Income|                        No|            Yes|     1|  5|            3|
| 34|             No|    Mid

In [5]:
df.columns

['Age',
 'FrequentWatcher',
 'AnnualIncomeClass',
 'AccountSyncedToSocialMedia',
 'BookedFoodOrNot',
 'Target',
 'Id',
 'ServicesOpted']

In [5]:
rows_to_convert = df.columns
rows_to_convert.remove("Id")
rows_to_convert.remove("Target")
rows_to_convert

['Age',
 'FrequentWatcher',
 'AnnualIncomeClass',
 'AccountSyncedToSocialMedia',
 'BookedFoodOrNot',
 'ServicesOpted']

In [6]:
",".join(rows_to_convert)

'Age,FrequentWatcher,AnnualIncomeClass,AccountSyncedToSocialMedia,BookedFoodOrNot,ServicesOpted'

In [7]:
df.groupBy("Target").count().show()

+------+-----+
|Target|count|
+------+-----+
|     0|  730|
|     1|  224|
+------+-----+



In [9]:
df.show()

+---+---------------+-----------------+--------------------------+---------------+------+---+-------------+
|Age|FrequentWatcher|AnnualIncomeClass|AccountSyncedToSocialMedia|BookedFoodOrNot|Target| Id|ServicesOpted|
+---+---------------+-----------------+--------------------------+---------------+------+---+-------------+
| 34|             No|    Middle Income|                        No|            Yes|     0|  0|            8|
| 34|            Yes|       Low Income|                       Yes|             No|     1|  1|            7|
| 37|             No|    Middle Income|                       Yes|             No|     0|  2|            5|
| 30|             No|    Middle Income|                        No|             No|     0|  3|            4|
| 30|             No|       Low Income|                        No|             No|     0|  4|            3|
| 27|            Yes|      High Income|                        No|            Yes|     1|  5|            3|
| 34|             No|    Mid

In [8]:
df.drop("Target").toPandas().to_csv("src/resources/data/movie_master.csv", header=True, index=False)

In [9]:
from src.utils.functions import get_row_as_text, hf_embeddings, get_ars_retrieved_df
train_df = get_row_as_text(df, rows_to_convert)

/home/dvadmin/look_alike_modelling_streamlit/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/dvadmin/look_alike_modelling_streamlit/venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
train_df.select("row_as_text").show(2, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------+
|row_as_text                                                                                                                           |
+--------------------------------------------------------------------------------------------------------------------------------------+
|Age: 34; FrequentWatcher: No; AnnualIncomeClass: Middle Income; AccountSyncedToSocialMedia: No; BookedFoodOrNot: Yes; ServicesOpted: 8|
|Age: 34; FrequentWatcher: Yes; AnnualIncomeClass: Low Income; AccountSyncedToSocialMedia: Yes; BookedFoodOrNot: No; ServicesOpted: 7  |
+--------------------------------------------------------------------------------------------------------------------------------------+
only showing top 2 rows



In [11]:
train_df.printSchema()

root
 |-- Age: string (nullable = true)
 |-- FrequentWatcher: string (nullable = true)
 |-- AnnualIncomeClass: string (nullable = true)
 |-- AccountSyncedToSocialMedia: string (nullable = true)
 |-- BookedFoodOrNot: string (nullable = true)
 |-- Target: string (nullable = true)
 |-- Id: long (nullable = false)
 |-- ServicesOpted: integer (nullable = true)
 |-- row_as_text: string (nullable = false)



In [12]:
step = 1500
k = 4000

texts_list = train_df.rdd.collect()
# texts_list = [x[0] for x in texts_label_list]
# texts_list
texts_list[0]

Row(Age='34', FrequentWatcher='No', AnnualIncomeClass='Middle Income', AccountSyncedToSocialMedia='No', BookedFoodOrNot='Yes', Target='0', Id=0, ServicesOpted=8, row_as_text='Age: 34; FrequentWatcher: No; AnnualIncomeClass: Middle Income; AccountSyncedToSocialMedia: No; BookedFoodOrNot: Yes; ServicesOpted: 8')

In [13]:
import os

# Specify the path of the directory you want to create
db_dir = "src/resources/embeddings/movie"

# Create the directory
os.makedirs(db_dir, exist_ok=True)

In [66]:
from langchain.vectorstores import Chroma
from src.utils.functions import hf_embeddings
import chromadb
import os

client = chromadb.PersistentClient(path=db_dir)

vdb = Chroma(persist_directory=db_dir, embedding_function=hf_embeddings,
                 collection_metadata={"hnsw:space": "cosine"}, client=client)

In [14]:
from langchain.vectorstores import Chroma
vdb = Chroma(persist_directory=db_dir, embedding_function=hf_embeddings, collection_metadata={"hnsw:space": "cosine"})
for i in range(0, len(texts_list), step):
    texts = [x.row_as_text for x in texts_list[i:i+step]]
    metadata = [{"Id": str(x.Id), "Target": str(x.Target)} for x in texts_list[i:i+step]]
    vdb.add_texts(texts, metadata)

vdb.persist()

In [ ]:
from langchain.vectorstores import Chroma
from src.utils.functions import hf_embeddings
import chromadb

client = chromadb.PersistentClient(path=db_dir)

vdb = Chroma(persist_directory=db_dir, embedding_function=hf_embeddings,
                 collection_metadata={"hnsw:space": "cosine"}, client=client)

In [15]:
vdb._collection.count()

954

24/08/26 13:37:22 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 2599989 ms exceeds timeout 120000 ms
24/08/26 13:37:23 WARN SparkContext: Killing executors is not supported by current scheduler.
24/08/26 13:37:25 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$

In [24]:
# db_dir

In [25]:
# train_df.filter(F.col("Target") == 1).drop("row_as_text").toPandas().to_csv("src/resources/data/movie_test.csv", header=True, index=False)

In [26]:
# spark.read.option("header", "true").csv("src/resources/data/movie_test.csv").show()

In [ ]:
df_input = train_df.drop("row_as_text")
df_input.show()

In [38]:
df_input = df_input.filter(F.col("Target") == 1).limit(10).union(df_input.filter(F.col("Target") == 0).limit(10))

In [39]:
df_input.show()

+---+---------------+-----------------+-------------+--------------------------+---------------+------+---+
|Age|FrequentWatcher|AnnualIncomeClass|ServicesOpted|AccountSyncedToSocialMedia|BookedFoodOrNot|Target| Id|
+---+---------------+-----------------+-------------+--------------------------+---------------+------+---+
| 34|            Yes|       Low Income|            5|                       Yes|             No|     1|  1|
| 27|            Yes|      High Income|            1|                        No|            Yes|     1|  5|
| 34|             No|       Low Income|            2|                       Yes|             No|     1|  7|
| 36|            Yes|      High Income|            1|                        No|             No|     1|  9|
| 28|             No|    Middle Income|            2|                        No|             No|     1| 11|
| 37|             No|       Low Income|            2|                       Yes|             No|     1| 19|
| 31|            Yes|      H

In [40]:
df_input.drop("Target").toPandas().to_csv("src/resources/data/movie_test.csv", header=True, index=False)

In [37]:
df_input.groupBy("Target").count().show()

+------+-----+
|Target|count|
+------+-----+
|     0|  730|
|     1|  224|
+------+-----+

